# Exploring the Adver-City Dataset 

Located [here](https://labs.cs.queensu.ca/quarrg/datasets/adver-city/).

In [1]:
# imports
import utils

The datafiles are formatted as follows:
`[prefix]_[weather]_[type].7z`

We will create dictionaries to map these valid values for easier access.

In [2]:
# various CARLA maps and scenario families
VALID_PREFIX = {
    "rcnj", "ri", "rsnj", "ui", "unj"
}

# weather types
VALID_WEATHER = {
    "cd": "clear day",
    "cn": "clear night",
    "fd": "fog day",
    "fn": "fog night",
    "hrd": "heavy rain day",
    "hrn": "heavy rain night",
    "srd": "soft rain day",
    "srn": "soft rain night",
    "gd": "glare day",
    "fhrd": "fog + heavy rain day",
    "fhrn": "fog + heavy rain night",
}

# density of the scene
VALID_DENSITY = {
    "s": "sparse",
    "d": "dense",
}

Download a file form the remote server

In [3]:
# path to where files are stored
DATA_DIR = "./data"
BASE_URL = "https://www.frdr-dfdr.ca/repo/files/1/published/publication_1079/submitted_data"

# specify which file (automate this later)
PREFIX = "rcnj"
WEATHER = "cn"
DENSITY = "s"

FILENAME = utils.build_filename(
    prefix=PREFIX,
    weather=WEATHER,
    density=DENSITY,
    VALID_PREFIX=VALID_PREFIX,
    VALID_WEATHER=VALID_WEATHER,
    VALID_DENSITY=VALID_DENSITY,
)

print('filename: ', FILENAME)

filename:  rcnj_cn_s.7z


Trying to download a sample file from the remote server to explore its contents.

In [ ]:
print('requesting', FILENAME, ' from remote server...') 
DOWNLOAD_URL = f"{BASE_URL}/{FILENAME}"
print('from url:', DOWNLOAD_URL)
FILE_PATH = utils.download_file(DATA_DIR, DOWNLOAD_URL, FILENAME)
print('file downloaded to', FILE_PATH)

requesting rcnj_cn_s.7z  from remote server...
from url: https://www.frdr-dfdr.ca/repo/files/1/published/publication_1079/submitted_data/rcnj_cn_s.7z
from url: https://www.frdr-dfdr.ca/repo/files/1/published/publication_1079/submitted_data/rcnj_cn_s.7z


2035705it [00:43, 47320.56it/s]

file downloaded to ./data/rcnj_cn_s.7z
